# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
F. Walter  ->  F. Walter  |  ['F. Walter']
A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
S. Vaughan  ->  S. Vaughan  |  ['S. Vaughan']
M. Zhang  ->  M. Zhang  |  ['M. Zhang']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
J. Shi  ->  J. Shi  |  ['J. Shi']


Arxiv has 71 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2508.08451
extracting tarball to tmp_2508.08451...

 done.


E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
F. Walter  ->  F. Walter  |  ['F. Walter']


Found 160 bibliographic references in tmp_2508.08451/main_v4_accepted.bbl.
Retrieving document from  https://arxiv.org/e-print/2508.08577


...

extracting tarball to tmp_2508.08720...

 done.
Retrieving document from  https://arxiv.org/e-print/2508.08766
extracting tarball to tmp_2508.08766...

 done.
Retrieving document from  https://arxiv.org/e-print/2508.08970


extracting tarball to tmp_2508.08970...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.08451-b31b1b.svg)](https://arxiv.org/abs/2508.08451) | **Quantification of The Age Dependence of Mid-Infrared Star Formation Rate Indicators**  |
|| D. Calzetti, et al. -- incl., <mark>E. Schinnerer</mark>, <mark>F. Walter</mark> |
|*Appeared on*| *2025-08-13*|
|*Comments*| *41 pages, 16 figures; accepted for publication on the Astrophysical Journal on August 8th, 2025*|
|**Abstract**|            We combine James Webb Space Telescope images of the nearby galaxy NGC 5194 in the hydrogen recombination line Pa-alpha (lambda=1.8756 micron) from the Cycle 1 program JWST-FEAST with 21 micron dust continuum images from the Cycle 2 Treasury program JWGT to quantify the difference in the calibration of mid-infrared star formation rates (SFR) between HII regions and galaxies. We use the archival HST H-alpha image to correct the Pa-alpha emission for the effects of dust attenuation. Our data confirm previous results that the dust-corrected Pa-alpha flux is tightly correlated with the 21 micron emission at the scales of HII regions. When combined with published JWST data for the HII regions of the galaxy NGC 628 and Spitzer 24 micron data for whole galaxies and for kpc-size galaxy regions, we show that the L(24)-L(Pa-alpha) correlation has exponent >1 across six decades in luminosity. In addition, the hybrid 24 micron+H-alpha SFR indicator has a scaling constant about 4.4 times higher for HII regions than for whole galaxies, also in agreement with previous results. Models of stellar populations with a range of star formation histories reveal that the observed trends can be entirely ascribed to and quantified with the contribution to the IR emission by stellar populations older than ~5-6 Myr. Based on the models' results, we provide: (1) a calibration for the infrared SFR across six orders of magnitude in L(24), from HII regions to luminous galaxies, and (2) a prescription for the scaling constant of the hybrid infrared SFR indicators as a function of the star formation timescale.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.08577-b31b1b.svg)](https://arxiv.org/abs/2508.08577) | **RUBIES spectroscopically confirms the high number density of quiescent galaxies from $\mathbf{2<z<5}$**  |
|| Y. Zhang, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2025-08-13*|
|*Comments*| *27 pages, 11 figures*|
|**Abstract**|            We present the number density of massive ($ \mathrm{ log (M_{*}/M_{\odot}) > 10.3} $) quiescent galaxies at $2<z<5$ using JWST NIRSpec PRISM spectra. This work relies on spectra from RUBIES, which provides excellent data quality and an unparalleled, well-defined targeting strategy to robustly infer physical properties and number densities. We identify quiescent galaxy candidates within RUBIES through principal component analysis and construct a final sample using star formation histories derived from spectro-photometric fitting of the NIRSpec PRISM spectra and NIRCam photometry. By inverting the RUBIES selection function, we correct for survey incompleteness and calculate the number density of massive quiescent galaxies at these redshifts, providing the most complete spectroscopic estimates prior to cosmic noon to date. We find that early massive quiescent galaxies are surprisingly common ($\gtrsim 10^{-5}$ Mpc$^{-3}$ by $4<z<5$), which is consistent with previous studies based on JWST photometry alone and/or in smaller survey areas. We compare our number densities with predictions from six state-of-the-art cosmological galaxy formation simulations. At $z>3$, most simulations fail to produce enough massive quiescent galaxies, suggesting the treatment of feedback and/or the channels for early efficient formation are incomplete in most galaxy evolution models.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.08720-b31b1b.svg)](https://arxiv.org/abs/2508.08720) | **Departures from Standard Disk Predictions in Intensive Ground-Based Monitoring of Three AGN**  |
|| D. Gonzalez-Buitrago, et al. -- incl., <mark>S. Vaughan</mark> |
|*Appeared on*| *2025-08-13*|
|*Comments*| *Accepted for publication in MNRAS. 32 pages, 12 figures, 5 tables*|
|**Abstract**|            We present ground-based, multi-band light curves of the AGN Mrk~509, NGC\,4151, and NGC\,4593 obtained contemporaneously with \sw\, monitoring. We measure cross-correlation lags relative to \sw\, UVW2 (1928~Å) and test the standard prediction for disk reprocessing, which assumes a geometrically thin, optically thick accretion disk where continuum interband delays follow the relation \( \tau(\lambda) \propto \lambda^{4/3} \). For Mrk~509 the 273-d \sw\, campaign gives well-defined lags that increase with wavelength as $\tau(\lambda)\propto\lambda^{2.17\pm0.2}$, steeper than the thin-disk prediction, and the optical lags are a factor of $\sim5$ longer than expected for a simple disk-reprocessing model. This ``disk-size discrepancy'' as well as excess lags in the $u$ and $r$ bands (which include the Balmer continuum and H$\alpha$, respectively) suggest a mix of short lags from the disk and longer lags from nebular continuum originating in the broad-line region. The shorter \sw\, campaigns, 69~d on NGC\,4151 and 22~d on NGC\,4593, yield less well-defined, shorter lags $<2$~d. The NGC\,4593 lags are consistent with $\tau(\lambda) \propto \lambda^{4/3}$ but with uncertainties too large for a strong test. For NGC\,4151 the \sw\, lags match $\tau(\lambda) \propto \lambda^{4/3}$, with a small $U$-band excess, but the ground-based lags in the $r$, $i$, and $z$ bands are significantly shorter than the $B$ and $g$ lags, and also shorter than expected from the thin-disk prediction. The interpretation of this unusual lag spectrum is unclear. Overall these results indicate significant diversity in the $\tau-\lambda$ relation across the optical/UV/NIR, which differs from the more homogeneous behavior seen in the \sw\, bands.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.08766-b31b1b.svg)](https://arxiv.org/abs/2508.08766) | **Solar magnetic flux rope eruptions caused by inverse flux feeding processes**  |
|| Q. Zhang, et al. -- incl., <mark>M. Zhang</mark>, <mark>Y. Wang</mark> |
|*Appeared on*| *2025-08-13*|
|*Comments*| *This manuscript has been accepted by Astronomy & Astrophysics*|
|**Abstract**|            Large-scale solar eruptions are generally accepted to have coronal magnetic flux ropes as their core structures. Recent studies found that the solar eruptions could be initiated by a sequence of flux feeding processes, during with chromospheric fibrils rise and merge with the pre-existing coronal flux rope. Further theoretical analyses have demonstrated that the normal flux feeding, i.e. the axial magnetic flux within the fibril is in the same direction as that in the flux rope, results in the accumulation of the total axial flux within the flux rope, so as to initiate the eruption. If the directions of the axial flux in the fibril and the flux rope are opposite, it is termed inverse flux feeding, whose influence on coronal flux ropes, however, is still unclear. In this paper, we use a 2.5-dimensional magnetohydrodynamic model to simulate the evolution of coronal flux ropes associated with inverse flux feeding. It is found that inverse flux feeding is also efficient in causing solar eruptions: although the total signed axial magnetic flux of the rope decreases after inverse flux feeding, the total unsigned axial flux can accumulate; the eruption occurs if the unsigned axial flux of the rope reaches a critical value, which is almost the same as the threshold for normal flux feeding. The total axial currents within the rope are also similar during the onset of the eruptions caused by both normal and inverse flux feeding. Our simulation results suggest that it is the unsigned axial magnetic flux rather than the signed axial flux that regulates the onset of coronal flux rope eruptions.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.08970-b31b1b.svg)](https://arxiv.org/abs/2508.08970) | **Measuring the Magnetic Field of a Coronal Mass Ejection from Low to Middle Corona**  |
|| X. Chen, et al. -- incl., <mark>J. Shi</mark> |
|*Appeared on*| *2025-08-13*|
|*Comments*| *15 pages, 7 figures*|
|**Abstract**|            A major challenge in understanding the initiation and evolution of coronal mass ejections (CMEs) is measuring the magnetic field of the magnetic flux ropes (MFRs) that drive CMEs. Recent developments in radio imaging spectroscopy have paved the way for diagnosing the CMEs' magnetic field using gyrosynchrotron radiation. We present magnetic field measurements of a CME associated with an X5-class flare by combining radio imaging spectroscopy data in microwaves (1--18 GHz) and meter-wave (20--88 MHz), obtained by the Owens Valley Radio Observatory's Expanded Owens Valley Solar Array (EOVSA) and Long Wavelength Array (OVRO-LWA), respectively. EOVSA observations reveal that the microwave source, observed in the low corona during the initiation phase of the eruption, outlines the bottom of the rising MFR-hosting CME bubble seen in extreme ultraviolet and expands as the bubble evolves. As the MFR erupts into the middle corona and appears as a white light CME, its meter-wave counterpart, observed by OVRO-LWA, displays a similar morphology. For the first time, using gyrosynchrotron spectral diagnostics, we obtain magnetic field measurements of the erupting MFR in both the low and middle corona, corresponding to coronal heights of 1.02 and 2.83 $R_{\odot}$. The magnetic field strength is found to be around 300 G at 1.02 $R_{\odot}$ during the CME initiation, and about 0.6 G near the leading edge of the CME when it propagates to 2.83 $R_{\odot}$. These results provide critical new insights into the magnetic structure of the CME and its evolution during the early stages of its eruption.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2508.08451/./ew_pa_new.png', 'tmp_2508.08451/./ebvpacor_new.png', 'tmp_2508.08451/./l21lpacor_new.png', 'tmp_2508.08451/./siglum24_pa_new.png', 'tmp_2508.08451/./lir_ebv.png', 'tmp_2508.08451/./sir_spa_new.png']
copying  tmp_2508.08451/./ew_pa_new.png to _build/html/
copying  tmp_2508.08451/./ebvpacor_new.png to _build/html/
copying  tmp_2508.08451/./l21lpacor_new.png to _build/html/
copying  tmp_2508.08451/./siglum24_pa_new.png to _build/html/
copying  tmp_2508.08451/./lir_ebv.png to _build/html/
copying  tmp_2508.08451/./sir_spa_new.png to _build/html/
exported in  _build/html/2508.08451.md
    + _build/html/tmp_2508.08451/./ew_pa_new.png
    + _build/html/tmp_2508.08451/./ebvpacor_new.png
    + _build/html/tmp_2508.08451/./l21lpacor_new.png
    + _build/html/tmp_2508.08451/./siglum24_pa_new.png
    + _build/html/tmp_2508.08451/./lir_ebv.png
    + _build/html/tmp_2508.08451/./sir_spa_new.png
found figures ['tmp_2508.08577/./figures/PCA_selection_v4.png', 'tmp_2508

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Quantification of The Age Dependence of Mid--Infrared Star Formation Rate Indicators

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2508.08451-b31b1b.svg)](https://arxiv.org/abs/2508.08451)<mark>Appeared on: 2025-08-13</mark> -  _41 pages, 16 figures; accepted for publication on the Astrophysical Journal on August 8th, 2025_

</div>
<div id="authors">

D. Calzetti, et al. -- incl., <mark>E. Schinnerer</mark>, <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** We combine James Webb Space Telescope images of the nearby galaxy NGC 5194  in the hydrogen recombination line Pa $\alpha$ ( $\lambda$ 1.8756 $\mu$ m) from the Cycle 1 program JWST--FEAST with 21 $\mu$ m dust continuum images from the Cycle 2 Treasury program JWGT to quantify the difference in the calibration of mid--infrared star formation rates (SFR) between HII regions and galaxies. We use the archival HST H $\alpha$ image to correct the Pa $\alpha$ emission for the effects of dust attenuation. Our data confirm previous results that the dust--corrected Pa $\alpha$ flux is tightly correlated with the 21 $\mu$ m emission at the scales of HII regions. When combined with published JWST data for the HII regions of the galaxy NGC 628 and Spitzer 24 $\mu$ m data for whole galaxies and for kpc--size galaxy regions, we show that the L(24)--L(Pa $\alpha$ ) correlation has exponent $>$ 1 across six decades in luminosity. In addition, the hybrid 24 $\mu$ m+H $\alpha$ SFR indicator has a scaling constant about 4.4 times higher for HII regions than for whole galaxies, also in agreement with previous results. Models of stellar populations with a range of star formation histories reveal that the observed trends can be entirely ascribed to and quantified with the contribution to the IR emission by stellar populations older than $\sim$ 5--6 Myr. Based on the models' results, we provide: (1) a calibration for the infrared SFR across six orders of magnitude in L(24), from HII regions to luminous galaxies, and (2) a prescription for the scaling constant of the hybrid infrared SFR indicators as a function of the star formation timescale.

</div>

<div id="div_fig1">

<img src="tmp_2508.08451/./ew_pa_new.png" alt="Fig3.1" width="50%"/><img src="tmp_2508.08451/./ebvpacor_new.png" alt="Fig3.2" width="50%"/>

**Figure 3. -** The EW(Pa$\alpha$) (left panel) and the color excess E(B$-$V) (right panel) as a function of the attenuation--corrected luminosity in Pa$\alpha$ for the HII regions in NGC 5194. The data are shown as black circles, with their 1$\sigma$ uncertainties. The right panel includes the data for NGC 628 as magenta triangles, with their 1$\sigma$ uncertainties. Both panels show as a vertical black dot--dashed line the expected Pa$\alpha$ luminosity of a 4 Myr old, 3,000 M$_{\odot}$ cluster, the lower-end luminosity limit in our analysis to mitigate stochastic IMF sampling effects  ([Cerviño, et. al 2002]()) . ** (Left):** The magenta dotted line shows the EW(Pa$\alpha$) model track of an HII region of constant mass $\sim$2$\times$10$^5$ M$_{\odot}$ with increasing age from 1 Myr (top--right) to 10 Myr (bottom--left). The red continuous line is the model track for a constant age, 3 Myr old, HII region of decreasing mass, from $\sim$3$\times$10$^5$ M$_{\odot}$ down to $\sim$2$\times$10$^2$ M$_{\odot}$,  immersed in a constant luminosity non--ionizing stellar field, with a scatter of factors $-$3/$+$5 (higher/lower red dashed lines) about the mean trend. The cyan line is the same track, but for a 6 Myr old HII region. The blue lines are the tracks for NGC 628,  from [Calzetti, Adamo and Linden (2024)](), using the same model but with a larger photometric aperture
(continuous line for the mean trend and dashed lines for the scatter). ** (Right):** The two empirical relations shown are from [Calzetti, Kennicutt and Engelbracht (2007)]() for $\sim$0.5 kpc  star forming regions (blue solid line for the mean trend and dashed lines for the 90--percentile) and from [Garn and Best (2010)]() for galaxies (red continuous line).  (*fig:corr_lum*)

</div>
<div id="div_fig2">

<img src="tmp_2508.08451/./l21lpacor_new.png" alt="Fig4.1" width="50%"/><img src="tmp_2508.08451/./siglum24_pa_new.png" alt="Fig4.2" width="50%"/>

**Figure 4. -** ** (Left:)** The luminosity at 21 $\mu$m, L(21), as a function of the attenuation--corrected luminosity at Pa$\alpha$ for the 254 line--emitting regions in NGC 5194 (black circles, this work) and the 143 regions in NGC 628 (magenta triangles, [Calzetti, Adamo and Linden (2024)](), with 1$\sigma$ uncertainties. The horizontal dashed line and grey region show the location of the 5$\sigma$ threshold at 21 $\mu$m in NGC 5194. The black and magenta lines show the best fits through the data with Pa$\alpha$ luminosity above the IMF sampling limit (vertical dot--dashed line) for the two galaxies separately, while the red line is for the data from the two galaxies combined. ** (Right:)** The luminosity surface density at 24 $\mu$m, $\Sigma$(24), as a function of the attenuation--corrected luminosity surface density at Pa$\alpha$ for the same HII regions in NGC 5194 (black circles) and NGC 628 (magenta triangles), also with 1$\sigma$ uncertainties, after converting the luminosity at 21 $\mu$m to 24 $\mu$m. The solid lines indicate the fits through the HII regions using the same color scheme as the left panel.  The data for the luminosity surface densities of 160 $\sim$0.5 kpc regions at $\sim$solar metallicity from [Calzetti, Kennicutt and Engelbracht (2007)]() are shown as blue pentagons with the best linear fit from those authors marked as a blue line. (*fig:l21_vs_lpa*)

</div>
<div id="div_fig3">

<img src="tmp_2508.08451/./lir_ebv.png" alt="Fig14.1" width="50%"/><img src="tmp_2508.08451/./sir_spa_new.png" alt="Fig14.2" width="50%"/>

**Figure 14. -** ** (Left:)** The fraction of bolometric luminosity emerging in the infrared, as a function of color excess E(B--V), for the following stellar population models: 3 Myr old instantaneous burst (black solid line); constant star formation over 100 Myr (blue lines); and constant star formation over 10 Gyr (red solid line). The effect of different extinction/attenuation curve choices are shown for the 100 Myr constant star formation model: starburst attenuation curve (blue solid) and the Milky Way (blue dashed), Large Magellanic Cloud (blue short dot--dashed) and Small Magellanic Cloud (blue long dot--dashed) extinction curves. ** (Right:)** The 24 $\mu$m luminosity surface density as a function of the extinction--corrected Pa$\alpha$ luminosity surface density for the three star formation models described in the Left panel. The Pa$\alpha$ surface density is derived via the E(B--V)--$\Sigma$(Pa$\alpha$) relation in equation \ref{equa:newebv} ([Calzetti, Kennicutt and Engelbracht 2007]()) . The models for the 3 Myr instantaneous burst (black solid line), $\tau$=100 Myr constant star formation (blue solid line) and $\tau$=10 Gyr constant star formation (red solid line) mark a sequence of increasing $\Sigma$(24) at fixed $\Sigma$(Pa$\alpha$)$_{corr}$. The relation at each age is shown with the 90\% scatter in the E(B--V)--Pa$\alpha$ relation from [Calzetti, Kennicutt and Engelbracht (2007)]() as dashed lines.  The black dot--dashed straight lines are examples of linear relations with slopes (bottom to top) of 1 and 1.14, to provide a visual reference. (*fig:LIR_EBV*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2508.08451"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# RUBIES spectroscopically confirms the high number density of quiescent galaxies from $\mathbf{2<z<5}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2508.08577-b31b1b.svg)](https://arxiv.org/abs/2508.08577)<mark>Appeared on: 2025-08-13</mark> -  _27 pages, 11 figures_

</div>
<div id="authors">

Y. Zhang, et al. -- incl., <mark>A. d. Graaff</mark>

</div>
<div id="abstract">

**Abstract:** We present the number density of massive ( $ \mathrm{ log (M_{*}/M_{\odot}) > 10.3} $ ) quiescent galaxies at $2<z<5$ using JWST NIRSpec PRISM spectra. This work relies on spectra from RUBIES, which provides excellent data quality and an unparalleled, well-defined targeting strategy to robustly infer physical properties and number densities. We identify quiescent galaxy candidates within RUBIES through principal component analysis and construct a final sample using star formation histories derived from spectro-photometric fitting of the NIRSpec PRISM spectra and NIRCam photometry. By inverting the RUBIES selection function, we correct for survey incompleteness and calculate the number density of massive quiescent galaxies at these redshifts, providing the most complete spectroscopic estimates prior to cosmic noon to date. We find that early massive quiescent galaxies are surprisingly common ( $\gtrsim 10^{-5}$ Mpc $^{-3}$ by $4<z<5$ ), which is consistent with previous studies based on JWST photometry alone and/or in smaller survey areas. We compare our number densities with predictions from six state-of-the-art cosmological galaxy formation simulations. At $z>3$ , most simulations fail to produce enough massive quiescent galaxies, suggesting the treatment of feedback and/or the channels for early efficient formation are incomplete in most galaxy evolution models.

</div>

<div id="div_fig1">

<img src="tmp_2508.08577/./figures/PCA_selection_v4.png" alt="Fig1" width="100%"/>

**Figure 1. -** Demonstration of the PCA analysis of NIRSpec/PRISM spectra from the DJA: The top row includes super color (SC) distributions (left and center) and derived eigenspectra (right panel). Each SC corresponds to the normalization of an eigenspectrum for each individual source, thus SC space location maps to spectral types. The bottom rows highlight representative examples: Dusty Star-Forming Galaxies (brown triangle), Little Red Dots (yellow circle), Old Quiescent Galaxies (red hexagon), Young Quiescent/Post-Starburst Galaxies (green star), Evolved/Napping Star-Forming Galaxies (blue cross), and Young Star-Forming Galaxies (purple cross). The de-redshifted original spectrum is shown in grey. The \texttt{SpectRes}-resampled spectrum used in PCA is shown as black dots.   (*fig:pca_selection*)

</div>
<div id="div_fig2">

<img src="tmp_2508.08577/./figures/diff_flavors_vs_obs.png" alt="Fig6" width="100%"/>

**Figure 6. -** Left panel: Different flavors of total quiescent galaxy number density binned by redshifts from this sample. We show the number density derived from the entire extended quiescent sample (pink hexagons), the entire strict quiescent sample (red hexagons), and the strict quiescent sample but using parent catalog sources from the entire field rather than just the survey footprints for completeness calculation (orange hexagons). Neither our quiescence criteria nor any potential survey targeting bias has a significant impact on the resulting number density. Middle panel: Our fiducial quiescent galaxy number density compared to literature values using JWST data. Our result is in agreement with previous JWST measurements overall. Right panel: Number densities of old ($t_{90} > 0.8 Gyr$) and young ($t_{90} < 0.8 Gyr$) quiescent galaxies in this work compared to (non-JWST) literature values. This work finds a higher number density for young quiescent galaxies at $3<z<4$ and for old quiescent galaxies at $2<z<3$ than previous non-JWST works.  (*fig:number-densities-observations*)

</div>
<div id="div_fig3">

<img src="tmp_2508.08577/./figures/SED_fits_12594+42328.png" alt="Fig2" width="100%"/>

**Figure 2. -** Example spectro-photometric fits of a $z\sim4$ post-starburst galaxy (ID: RUBIES-UDS-12594) and an old quiescent galaxy (ID: RUBIES-EGS-42328). For each row, the upper left panel shows the SED of observed NIRCam photometry and uncertainty (orange circle and error bar), observed NIRSpec PRISM spectrum and uncertainty (red solid line and pink bands), best-fit model photometry and its 68\% confidence interval (black square and errorbar), and best-fit model spectrum and its 68\% confidence interval (black solid line and grey band). The bottom left panel shows the residuals of our fits to the observed photometry (orange squares) and spectrum (red solid line). The inset shows the NIRCam/F444W image of the target galaxy. The red rectangle traces the central MSA micro-shutter used to compute slit-like aperture photometry, and the red circle traces the circular photometric catalog aperture. The right panels show the median (lines) and $16-84\%$ intervals (bands) for the inferred SFHs and $t_90$ measurements. These fits allow us to robustly determine the physical properties of these galaxies. (*fig:SED_young_QG*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2508.08577"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

133  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
